In [8]:
!pip install google-cloud-aiplatform google-cloud-bigquery pandas scikit-learn joblib xgboost --upgrade

print("Libraries installed!")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Libraries installed!


In [13]:
# ------------------------------------------------------------------
# CELL 2: Imports and Configuration
# ------------------------------------------------------------------
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, roc_auc_score, average_precision_score, brier_score_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.calibration import calibration_curve

# --- !!! PRODUCTION-GRADE IMPORTS !!! ---
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
# -----------------------------------------

from google.cloud import bigquery
from google.cloud import aiplatform
import numpy as np
import xgboost as xgb
from sklearn.svm import SVC

import json

# --- !!! YOUR PROJECT DETAILS !!! ---
PROJECT_ID = "artful-affinity-476513-t7"
BQ_DATASET = "complete_db"
REGION = "us-central1"
MODEL_DISPLAY_NAME = "disease_outbreak_prediction_model"
# -------------------------------------

# Initialize the Google Cloud clients
bq_client = bigquery.Client(project=PROJECT_ID)
aiplatform.init(project=PROJECT_ID, location=REGION)

print(f"Clients initialized for project {PROJECT_ID}. Ready to load data.")

Clients initialized for project artful-affinity-476513-t7. Ready to load data.


In [ ]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime_dtype

DISTRICT_RENAME_MAP = {
    'District': 'district',
    'district_name': 'district',
    'DistrictName': 'district',
    'Population': 'total_population',
    'population': 'total_population',
    'Patient Inflow (Daily)': 'patient_inflow_daily',
    'patient_inflow_daily': 'patient_inflow_daily',
    'Disease Outbreak': 'disease_outbreak',
    'disease_outbreak': 'disease_outbreak',
    'last_updated': 'last_updated',
    'last_inspection_date': 'last_inspection_date',
    'request_date': 'request_date',
    'report_date': 'report_date',
    'total_population': 'total_population'
}

def norm_dist(series):
    return (
        series.astype(str)
              .str.strip()
              .str.lower()
              .str.replace(r"\s+", " ", regex=True)
    )

def norm_week(timestamp_series):
    ts = pd.to_datetime(timestamp_series, errors='coerce')
    try:
        ts = ts.dt.tz_localize(None)
    except (AttributeError, TypeError):
        try:
            ts = ts.dt.tz_convert(None)
        except Exception:
            pass
    return ts.dt.to_period('W-MON').dt.start_time

def find_date_col(df, candidates=None):
    if df is None or df.empty:
        return None
    if candidates is None:
        candidates = (
            'report_date',
            'request_date',
            'last_updated',
            'inspection_date',
            'last_inspection_date',
            'resolution_date',
            'date',
            'event_date'
        )
    for col in candidates:
        if col in df.columns:
            return col
    for col in df.columns:
        lowered = col.lower()
        if 'date' in lowered or 'time' in lowered or 'updated' in lowered:
            return col
    for col in df.columns:
        try:
            if is_datetime_dtype(df[col]):
                return col
        except Exception:
            continue
    return None

def normalize_tables():
    for var in ['df_health','df_roads','df_safety','df_services','df_env','df_agri','df_pop']:
        df = globals().get(var)
        if df is None or df.empty:
            continue
        rename_candidates = {c: DISTRICT_RENAME_MAP[c] for c in df.columns if c in DISTRICT_RENAME_MAP}
        if rename_candidates:
            df = df.rename(columns=rename_candidates)
        if 'district' in df.columns and 'district_norm' not in df.columns:
            df['district_norm'] = norm_dist(df['district'])
        date_col = find_date_col(df)
        if date_col:
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        globals()[var] = df

print('Normalization helpers ready. Call normalize_tables() after loading raw tables.')

Schema normalization complete. Check printed warnings and df_* column lists if any.


In [ ]:
TABLE_MAP = {
    'df_health': 'ai_governance_health_facilities',
    'df_roads': 'ai_governance_infrastructure_roads',
    'df_safety': 'ai_governance_public_safety_reports',
    'df_services': 'ai_governance_citizen_services_requests',
    'df_env': 'ai_governance_environment_monitoring',
    'df_agri': 'ai_governance_agriculture_insights',
    'df_pop': 'ai_governance_population_demographics',
}

# Optional: limit rows for quick iteration (set to None to load full table)
ROW_LIMIT = None  # e.g., 20000 or None

for varname, table in TABLE_MAP.items():
    fq_table = f"{PROJECT_ID}.{BQ_DATASET}.{table}"
    try:
        print(f"Loading `{fq_table}` -> {varname} ...")
        if ROW_LIMIT:
            sql = f"SELECT * FROM `{fq_table}` LIMIT {ROW_LIMIT}"
        else:
            sql = f"SELECT * FROM `{fq_table}`"
        job = bq_client.query(sql)
        df = job.to_dataframe()   # may take time for big tables
        globals()[varname] = df
        print(f"Loaded {varname}: shape={df.shape}")
    except Exception as e:
        print(f"Failed to load `{fq_table}` into {varname}: {e}")

normalize_tables()
print('Applied normalization to loaded tables (district_norm + datetime coercion).')

# Quick peek
for var in TABLE_MAP.keys():
    if var in globals() and getattr(globals()[var], "shape", (0,0))[0] > 0:
        print(f"\n{var} sample (first 3 rows):")
        display(globals()[var].head(3))
    else:
        print(f"\n{var} is empty or not found (shape={getattr(globals().get(var), 'shape', None)})")

Loading `artful-affinity-476513-t7.complete_db.ai_governance_health_facilities` -> df_health ...
Loaded df_health: shape=(221, 9)
Loading `artful-affinity-476513-t7.complete_db.ai_governance_infrastructure_roads` -> df_roads ...
Loaded df_roads: shape=(299, 13)
Loading `artful-affinity-476513-t7.complete_db.ai_governance_public_safety_reports` -> df_safety ...
Loaded df_safety: shape=(53, 12)
Loading `artful-affinity-476513-t7.complete_db.ai_governance_citizen_services_requests` -> df_services ...
Loaded df_services: shape=(93, 13)
Loading `artful-affinity-476513-t7.complete_db.ai_governance_environment_monitoring` -> df_env ...
Loaded df_env: shape=(72, 16)
Loading `artful-affinity-476513-t7.complete_db.ai_governance_agriculture_insights` -> df_agri ...
Loaded df_agri: shape=(78, 16)
Loading `artful-affinity-476513-t7.complete_db.ai_governance_population_demographics` -> df_pop ...
Loaded df_pop: shape=(100, 16)

df_health sample (first 3 rows):


,hospital_name,district,patient_inflow_daily,disease_outbreak,vaccination_coverage,staff_count,hospital_type,emergency_services,report_date
0,Apex Hospital,Gondia,100,None,69,60,Private,True,2024-11-09
1,Satara District Hospital,Satara,160,None,78,110,Public,True,2025-07-25
2,Lifeline Hospital,Aurangabad,170,None,80,130,Public,True,2025-09-23



df_roads sample (first 3 rows):


,road_id,district,road_name,construction_year,last_maintenance_year,condition_score,repair_requests_last_quarter,estimated_repair_cost,status,road_length_km,traffic_volume_daily,surface_type,last_maintenance_date
0,ROAD-035,Mumbai Suburban,SV Road,1990,2024,8.2,3,600000,Good,15.0,80000.0,Concrete,2024-01-01
1,ROAD-150,Pune,Chakan Shikrapur Road,2003,2024,8.2,3,600000,Good,25.0,40000.0,Concrete,2024-01-01
2,ROAD-243,Mumbai Suburban,Bandra-Worli Sea Link,2009,2024,9.0,3,500000,Good,6.0,90000.0,Concrete,2024-01-01



df_safety sample (first 3 rows):


...

,station_id,district,location_name,air_quality_index,pm25_level,pm10_level,co_level,noise_level_db,waste_collection_efficiency,water_quality_index,last_inspection_date,status,monitoring_type,dominant_pollutant,vegetation_cover_percentage,renewable_energy_usage_percent
0,ENV-061,Gadchiroli,Etapalli Region,42.0,11.0,24.0,0.1,38.0,99.0,99.0,2025-10-29,Good,Water,None,87.0,52.0
1,ENV-028,Gadchiroli,Desaiganj Forest,45.0,12.0,25.0,0.1,40.0,98.0,98.0,2025-10-29,Good,Water,None,85.0,50.0
2,ENV-058,Wardha,Samudrapur Rural,48.0,14.0,28.0,0.1,45.0,99.0,97.0,2025-10-30,Good,All,None,82.0,48.0



df_agri sample (first 3 rows):


,farm_id,district,crop_type,farm_area_hectares,soil_quality_score,average_yield_tonnes,irrigation_source,rainfall_mm_last_month,fertilizer_usage_kg_per_hectare,pest_incidents_last_season,crop_health_index,last_updated,farmer_name,farming_method,market_price_per_tonne_inr,crop_variety
0,FARM-036,Nashik,Wheat,5.0,7.0,3.8,Canal,105.0,80.0,2,7.7,2025-10-29,Rahul Joshi,Conventional,29000.0,Raj-3077
1,FARM-052,Osmanabad,Wheat,5.5,6.8,3.9,Canal,95.0,84.0,2,7.7,2025-10-28,Sanjay Jagtap,Conventional,29500.0,HI-8663
2,FARM-060,Beed,Wheat,6.5,6.9,4.0,Canal,92.0,86.0,2,7.8,2025-10-28,Priyanka Thorat,Conventional,30000.0,NI-5439



df_pop sample (first 3 rows):


,district_id,district,total_population,urban_population_percentage,rural_population_percentage,male_population,female_population,literacy_rate,employment_rate,youth_population_percentage,senior_population_percentage,avg_household_size,population_density_per_sqkm,last_census_year,projected_growth_rate,last_updated
0,DIST-075,Mangalwedha,9267296,78.4,21.6,4733369,4533927,85.7,55.6,23.9,6.5,3.8,4617.3,2021,1.8,2025-10-28
1,DIST-049,Dombivli,4281658,75.0,25.0,2218163,2063495,73.7,61.8,22.7,9.2,3.8,6197.3,2021,2.4,2025-10-28
2,DIST-100,Niphad,10107905,39.2,60.8,5105491,5002414,70.9,59.5,19.7,10.4,3.9,8633.0,2021,1.5,2025-10-28


In [ ]:
def first_existing(df, candidates):
    for column in candidates:
        if column in df.columns:
            return column
    return None

def parse_crime_reports(value):
    if pd.isna(value):
        return {}
    if isinstance(value, str):
        try:
            parsed = json.loads(value)
        except Exception:
            return {}
    elif isinstance(value, dict):
        parsed = [value]
    else:
        parsed = value
    if not isinstance(parsed, (list, tuple)):
        return {}
    counts = {}
    for item in parsed:
        if not isinstance(item, dict):
            continue
        crime_type = item.get('type')
        count = item.get('count', 1)
        try:
            count = int(float(count))
        except Exception:
            count = 0
        if crime_type and count:
            counts[crime_type] = counts.get(crime_type, 0) + count
    return counts

for suffix in ['health', 'env', 'safety', 'services', 'roads', 'agri', 'pop']:
    var_name = f'df_{suffix}'
    if var_name not in globals():
        globals()[var_name] = pd.DataFrame()

date_cols = {}
for suffix in ['health', 'env', 'safety', 'services', 'roads', 'agri']:
    df = globals()[f'df_{suffix}']
    if not df.empty and 'district_norm' not in df.columns and 'district' in df.columns:
        df['district_norm'] = norm_dist(df['district'])
    date_col = find_date_col(df)
    date_cols[suffix] = date_col
    if date_col:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    globals()[f'df_{suffix}'] = df

if not df_pop.empty and 'district_norm' not in df_pop.columns and 'district' in df_pop.columns:
    df_pop['district_norm'] = norm_dist(df_pop['district'])

# Health weekly aggregation
health_week = pd.DataFrame()
if not df_health.empty and date_cols.get('health'):
    health_date = date_cols['health']
    patient_col = first_existing(df_health, ['patient_inflow_daily', 'patient_inflow', 'patient_inflow_mean'])
    outbreak_col = first_existing(df_health, ['disease_outbreak'])
    health_cols = ['district_norm', health_date]
    if patient_col:
        health_cols.append(patient_col)
    if outbreak_col:
        health_cols.append(outbreak_col)
    h = df_health[health_cols].copy()
    h = h.dropna(subset=['district_norm'])
    h['week_start'] = norm_week(h[health_date])
    h = h.dropna(subset=['week_start'])
    h['health_events'] = 1
    agg_map = {'health_events': 'sum'}
    if patient_col:
        h['patient_inflow_value'] = pd.to_numeric(h[patient_col], errors='coerce')
        agg_map['patient_inflow_value'] = 'mean'
    if outbreak_col:
        outbreak_series = h[outbreak_col].astype(str).str.strip().str.lower()
        h['outbreak_flag'] = (~outbreak_series.isin({'', 'none', 'null', 'nan'})).astype(int)
        agg_map['outbreak_flag'] = 'sum'
    health_week = h.groupby(['district_norm', 'week_start']).agg(agg_map).reset_index()
    if 'patient_inflow_value' in health_week.columns:
        health_week = health_week.rename(columns={'patient_inflow_value': 'patient_inflow_mean'})
    if 'outbreak_flag' in health_week.columns:
        health_week = health_week.rename(columns={'outbreak_flag': 'outbreak_count'})
    else:
        health_week['outbreak_count'] = 0
    if 'health_events' not in health_week.columns:
        health_week['health_events'] = 0

# Environment weekly aggregation
env_week = pd.DataFrame()
if not df_env.empty and date_cols.get('env'):
    env_date = date_cols['env']
    env_metrics = [c for c in ['air_quality_index', 'pm25_level', 'pm10_level', 'water_quality_index', 'waste_collection_efficiency'] if c in df_env.columns]
    if env_metrics:
        env_cols = ['district_norm', env_date] + env_metrics
        e = df_env[env_cols].copy()
        e = e.dropna(subset=['district_norm'])
        e['week_start'] = norm_week(e[env_date])
        e = e.dropna(subset=['week_start'])
        agg_map = {c: 'mean' for c in env_metrics}
        env_week = e.groupby(['district_norm', 'week_start']).agg(agg_map).reset_index()

# Services weekly aggregation
services_week = pd.DataFrame()
if not df_services.empty and date_cols.get('services'):
    services_date = date_cols['services']
    service_cols = ['district_norm', services_date]
    if 'service_type' in df_services.columns:
        service_cols.append('service_type')
    svc = df_services[service_cols].copy()
    svc = svc.dropna(subset=['district_norm'])
    svc['week_start'] = norm_week(svc[services_date])
    svc = svc.dropna(subset=['week_start'])
    svc['services_events'] = 1
    agg_map = {'services_events': 'sum'}
    if 'service_type' in svc.columns:
        agg_map['service_type'] = 'count'
    services_week = svc.groupby(['district_norm', 'week_start']).agg(agg_map).reset_index()
    if 'service_type' in services_week.columns:
        services_week = services_week.rename(columns={'service_type': 'complaint_count'})

# Safety weekly aggregation from crime JSON
safety_week = pd.DataFrame()
if not df_safety.empty and date_cols.get('safety') and 'crime_reports' in df_safety.columns:
    safety_date = date_cols['safety']
    safe = df_safety[['district_norm', safety_date, 'crime_reports']].copy()
    safe = safe.dropna(subset=['district_norm'])
    safe['week_start'] = norm_week(safe[safety_date])
    safe = safe.dropna(subset=['week_start'])
    safe['crime_counts'] = safe['crime_reports'].apply(parse_crime_reports)
    crime_rows = []
    for _, row in safe.iterrows():
        counts = row['crime_counts'] or {}
        for crime_type, count in counts.items():
            crime_rows.append({'district_norm': row['district_norm'], 'week_start': row['week_start'], 'crime_type': crime_type, 'count': count})
    if crime_rows:
        crime_df = pd.DataFrame(crime_rows)
        safety_week = (
            crime_df.groupby(['district_norm', 'week_start', 'crime_type'])['count']
            .sum()
            .unstack(fill_value=0)
            .reset_index()
        )

for label, wk in [('health', health_week), ('env', env_week), ('services', services_week), ('safety', safety_week)]:
    if wk is None or wk.empty:
        print(f'{label}_week: empty')
    else:
        print(
            f"{label}_week rows={len(wk)}, districts={wk['district_norm'].nunique()}, "
            f"weeks={wk['week_start'].min()}->{wk['week_start'].max()}"
        )

weekly_tables = [health_week, env_week, services_week, safety_week]
signal_districts = set()
for wk in weekly_tables:
    if wk is not None and not wk.empty:
        signal_districts.update(wk['district_norm'].dropna().unique())

if df_pop.empty:
    raise ValueError('Population table is required to build the panel.')

if 'district_norm' not in df_pop.columns and 'district' in df_pop.columns:
    df_pop['district_norm'] = norm_dist(df_pop['district'])

pop_districts = set(df_pop['district_norm'].dropna().unique())
missing_in_pop = sorted(signal_districts - pop_districts)
if missing_in_pop:
    preview = ', '.join(missing_in_pop[:8])
    print(f'Districts with weekly signal missing in population ({len(missing_in_pop)}): {preview}')
missing_signal = sorted(pop_districts - signal_districts)
print(f'Districts with population but no weekly signal: {len(missing_signal)}')

districts = sorted(signal_districts & pop_districts)
if not districts:
    raise ValueError('No overlapping districts between signal tables and population table after normalization.')

week_bounds = []
for wk in weekly_tables:
    if wk is not None and not wk.empty:
        week_bounds.append((wk['week_start'].min(), wk['week_start'].max()))
if not week_bounds:
    raise ValueError('No weekly aggregates available to build panel.')

min_date = min(bound[0] for bound in week_bounds)
max_date = max(bound[1] for bound in week_bounds)
week_starts = pd.date_range(start=min_date, end=max_date, freq='W-MON')
week_starts = pd.Series(week_starts).dt.floor('D')

panel = pd.MultiIndex.from_product([districts, week_starts], names=['district_norm', 'week_start']).to_frame(index=False)

def merge_weekly(base, weekly):
    if weekly is None or weekly.empty:
        return base
    weekly = weekly.copy()
    weekly['week_start'] = pd.to_datetime(weekly['week_start']).dt.floor('D')
    return base.merge(weekly, on=['district_norm', 'week_start'], how='left')

panel['week_start'] = pd.to_datetime(panel['week_start']).dt.floor('D')
panel = merge_weekly(panel, health_week)
panel = merge_weekly(panel, env_week)
panel = merge_weekly(panel, services_week)
panel = merge_weekly(panel, safety_week)

pop_cols = [c for c in ['district_norm', 'district', 'total_population', 'population_density_per_sqkm', 'avg_household_size'] if c in df_pop.columns]
pop_frame = df_pop[pop_cols].drop_duplicates(subset=['district_norm'])
panel = panel.merge(pop_frame, on='district_norm', how='left')

if 'district' not in panel.columns:
    panel['district'] = panel['district_norm']

if 'total_population' in panel.columns:
    panel['total_population'] = pd.to_numeric(panel['total_population'], errors='coerce')
    panel['pop_per_100k'] = panel['total_population'].replace({0: np.nan}) / 100000.0
else:
    panel['pop_per_100k'] = np.nan

for col in ['outbreak_count', 'health_events', 'services_events', 'complaint_count']:
    if col in panel.columns:
        panel[col] = pd.to_numeric(panel[col], errors='coerce').fillna(0)

crime_cols = [c for c in panel.columns if c.startswith('crime_')]
for col in crime_cols:
    panel[col] = pd.to_numeric(panel[col], errors='coerce').fillna(0)
    panel[col] = np.where(panel['pop_per_100k'] > 0, panel[col] / panel['pop_per_100k'], 0)

cont_candidates = [
    'patient_inflow_mean',
    'air_quality_index',
    'pm25_level',
    'pm10_level',
    'water_quality_index',
    'waste_collection_efficiency',
    'traffic_volume_daily',
    'condition_score'
 ]
cont_cols = [c for c in cont_candidates if c in panel.columns]
dropped_cont_cols = []
for col in cont_cols:
    panel[col] = pd.to_numeric(panel[col], errors='coerce')
    panel[col] = panel.groupby('district_norm')[col].transform(lambda s: s.ffill().bfill())
    if panel[col].notna().sum() == 0:
        panel.drop(columns=[col], inplace=True)
        dropped_cont_cols.append(col)
        continue
    median_val = panel[col].median(skipna=True)
    if pd.isna(median_val):
        panel.drop(columns=[col], inplace=True)
        dropped_cont_cols.append(col)
        continue
    panel[col] = panel[col].fillna(median_val)

if dropped_cont_cols:
    print(f'Dropped continuous columns with no usable coverage: {dropped_cont_cols}')

for window in [1, 2, 4]:
    if 'pm25_level' in panel.columns:
        panel[f'pm25_roll_{window}w'] = panel.groupby('district_norm')['pm25_level'].transform(lambda s: s.shift(1).rolling(window, min_periods=1).mean())
    if 'patient_inflow_mean' in panel.columns:
        panel[f'patient_inflow_roll_{window}w'] = panel.groupby('district_norm')['patient_inflow_mean'].transform(lambda s: s.shift(1).rolling(window, min_periods=1).mean())

def future_sum(series, horizon):
    values = series.fillna(0).to_numpy()
    out = np.zeros(len(values), dtype=float)
    for idx in range(len(values)):
        start = idx + 1
        end = min(idx + 1 + horizon, len(values))
        out[idx] = values[start:end].sum()
    return pd.Series(out, index=series.index)

if 'outbreak_count' not in panel.columns:
    panel['outbreak_count'] = 0
panel['outbreak_count'] = pd.to_numeric(panel['outbreak_count'], errors='coerce').fillna(0)

HORIZON_WEEKS = 2
panel = panel.sort_values(['district_norm', 'week_start']).reset_index(drop=True)
panel['outbreak_next_14d'] = (
    panel.groupby('district_norm')['outbreak_count']
         .transform(lambda s: future_sum(s, HORIZON_WEEKS))
         .gt(0)
         .astype(int)
 )

def inner_hits(base, weekly):
    if weekly is None or weekly.empty:
        return 0
    pairs = weekly[['district_norm', 'week_start']].dropna().drop_duplicates()
    pairs['week_start'] = pd.to_datetime(pairs['week_start']).dt.floor('D')
    return (
        base[['district_norm', 'week_start']]
        .merge(pairs, on=['district_norm', 'week_start'], how='inner')
        .shape[0]
    )

print('Panel districts:', len(districts))
print('Panel weeks:', len(week_starts))
print('Rows with outbreak_count>0:', int((panel['outbreak_count'] > 0).sum()))
print('Positive rate (outbreak_next_14d):', panel['outbreak_next_14d'].mean())
print('Inner hits with health_week:', inner_hits(panel, health_week))
print('Inner hits with env_week:', inner_hits(panel, env_week))
print('Inner hits with services_week:', inner_hits(panel, services_week))
print('Inner hits with safety_week:', inner_hits(panel, safety_week))
if not env_week.empty:
    env_share = (
        panel[['district_norm', 'week_start']]
        .merge(env_week[['district_norm', 'week_start']].drop_duplicates(), how='inner')
        .shape[0] / len(panel)
    )
    print(f'Share of rows with raw env coverage: {env_share:.3f}')
else:
    print('Share of rows with raw env coverage: 0.000')

df_model = panel.copy()
df_model.to_csv('district_week_panel_demo.csv', index=False)
print('Saved CLEAN district_week_panel_demo.csv (normalized panel).')

Inferred date columns: {'health': 'report_date', 'roads': 'last_maintenance_date', 'safety': 'last_updated', 'services': 'request_date', 'env': 'last_inspection_date', 'agri': 'last_updated'}
Panel date range: 2024-01-03 00:00:00 to 2025-11-02 00:00:00
Health weekly rows: 215
Weeks with outbreak_count>0: 110
patient_inflow_mean coverage: 1.0
Panel shape: (9500, 34)
     district week_start  patient_inflow_mean  outbreak_count  health_events  \
0  Ahmednagar 2024-01-08                 <NA>             0.0            0.0   
1  Ahmednagar 2024-01-15                 <NA>             0.0            0.0   
2  Ahmednagar 2024-01-22                 <NA>             0.0            0.0   
3  Ahmednagar 2024-01-29                 <NA>             0.0            0.0   
4  Ahmednagar 2024-02-05                 <NA>             0.0            0.0   

   air_quality_index  pm25_level  pm10_level  water_quality_index  \
0                NaN         NaN         NaN                  NaN   
1            

In [41]:
# 1) Compare district vocabularies
print("pop districts (sample):", df_pop['district'].dropna().unique()[:10])
print("health districts (sample):", df_health['district'].dropna().unique()[:10])

# 2) How many overlaps?
pop_set = set(df_pop['district'].dropna().str.strip().str.lower())
health_set = set(df_health['district'].dropna().str.strip().str.lower())
print("overlap count:", len(pop_set & health_set))


pop districts (sample): ['Mangalwedha' 'Dombivli' 'Niphad' 'Chakan' 'Yeola' 'Badlapur' 'Panhala'
 'Kalwan' 'Ambegaon' 'Dodamarg']
health districts (sample): ['Gondia' 'Satara' 'Aurangabad' 'Ahmednagar' 'Sindhudurg' 'Parbhani'
 'Latur' 'Nanded' 'Dhule' 'Nashik']
overlap count: 31


In [38]:
# Inspect health / env columns & dtypes
print("df_health columns:", df_health.columns.tolist())
print(df_health.dtypes)
display(df_health.head(5))

print("df_env columns:", df_env.columns.tolist())
print(df_env.dtypes)
display(df_env.head(5))

cands = ['patient_inflow','patient_inflow_daily','Patient Inflow (Daily)','patient_inflow_mean']
for c in cands:
    if c in df_health.columns:
        print(c, "non-null fraction:", df_health[c].notna().mean(), "dtype:", df_health[c].dtype)
        
for c in ['disease_outbreak','Disease Outbreak','diseaseOutbreak']:
    if c in df_health.columns:
        print(c, "unique values:", df_health[c].astype(str).value_counts(dropna=False).head(10))
        
def coverage(panel, cols):
    return panel.groupby('district')[cols].apply(lambda s: s.notna().mean()).head(10)
print("Coverage sample (env + health):")
print(coverage(panel, [c for c in ['pm25_level','patient_inflow_mean','air_quality_index'] if c in panel.columns]))
print("Outbreak positives:", panel['outbreak_next_14d'].sum(), " / ", len(panel))

df_health columns: ['hospital_name', 'district', 'patient_inflow_daily', 'disease_outbreak', 'vaccination_coverage', 'staff_count', 'hospital_type', 'emergency_services', 'report_date']
hospital_name                   object
district                        object
patient_inflow_daily             Int64
disease_outbreak                object
vaccination_coverage             Int64
staff_count                      Int64
hospital_type                   object
emergency_services             boolean
report_date             datetime64[ns]
dtype: object


,hospital_name,district,patient_inflow_daily,disease_outbreak,vaccination_coverage,staff_count,hospital_type,emergency_services,report_date
0,Apex Hospital,Gondia,100,None,69,60,Private,True,2024-11-09
1,Satara District Hospital,Satara,160,None,78,110,Public,True,2025-07-25
2,Lifeline Hospital,Aurangabad,170,None,80,130,Public,True,2025-09-23
3,City Medical Center,Ahmednagar,170,None,81,100,Private,False,2025-09-09
4,New Vision Clinic,Ahmednagar,160,None,82,90,Private,False,2025-04-05


df_env columns: ['station_id', 'district', 'location_name', 'air_quality_index', 'pm25_level', 'pm10_level', 'co_level', 'noise_level_db', 'waste_collection_efficiency', 'water_quality_index', 'last_inspection_date', 'status', 'monitoring_type', 'dominant_pollutant', 'vegetation_cover_percentage', 'renewable_energy_usage_percent']
station_id                                object
district                                  object
location_name                             object
air_quality_index                        float64
pm25_level                               float64
pm10_level                               float64
co_level                                 float64
noise_level_db                           float64
waste_collection_efficiency              float64
water_quality_index                      float64
last_inspection_date              datetime64[ns]
status                                    object
monitoring_type                           object
dominant_pollutant            

,station_id,district,location_name,air_quality_index,pm25_level,pm10_level,co_level,noise_level_db,waste_collection_efficiency,water_quality_index,last_inspection_date,status,monitoring_type,dominant_pollutant,vegetation_cover_percentage,renewable_energy_usage_percent
0,ENV-061,Gadchiroli,Etapalli Region,42.0,11.0,24.0,0.1,38.0,99.0,99.0,2025-10-29,Good,Water,None,87.0,52.0
1,ENV-028,Gadchiroli,Desaiganj Forest,45.0,12.0,25.0,0.1,40.0,98.0,98.0,2025-10-29,Good,Water,None,85.0,50.0
2,ENV-058,Wardha,Samudrapur Rural,48.0,14.0,28.0,0.1,45.0,99.0,97.0,2025-10-30,Good,All,None,82.0,48.0
3,ENV-025,Wardha,Sevagram Rural,50.0,15.0,30.0,0.2,48.0,97.0,96.0,2025-10-30,Good,All,None,80.0,45.0
4,ENV-022,Ratnagiri,Ganpatipule Beach,55.0,18.0,35.0,0.2,50.0,96.0,95.0,2025-10-29,Good,Water,None,75.0,40.0


patient_inflow_daily non-null fraction: 1.0 dtype: Int64
disease_outbreak unique values: disease_outbreak
None        109
COVID-19     20
Dengue       20
Typhoid      20
Malaria      19
Cholera      17
Measles      16
Name: count, dtype: int64
Coverage sample (env + health):
            pm25_level  patient_inflow_mean  air_quality_index
district                                                      
Ahmednagar         1.0                  1.0                1.0
Ajra               1.0                  1.0                1.0
Akola              1.0                  1.0                1.0
Alandi             1.0                  1.0                1.0
Alibag             1.0                  1.0                1.0
Ambegaon           1.0                  1.0                1.0
Ambernath          1.0                  1.0                1.0
Amravati           1.0                  1.0                1.0
Aurangabad         1.0                  1.0                1.0
Badlapur           1.0         

In [ ]:
# ------------------------------------------------------------------
 # CELL 4.1: Create forward-looking target & windowed features (Horizon)
 # ------------------------------------------------------------------
 import warnings
 from datetime import timedelta

 HORIZON_DAYS = 14  # predict outbreak within next 14 days

 # 1) Forward-looking target from health events
 date_col = find_date_col(df_health)
 outbreak_col = first_existing(df_health, ['disease_outbreak', 'Disease Outbreak'])
 if date_col is None or outbreak_col is None:
     warnings.warn('Health table lacks a usable date or outbreak column; skipping forward-looking label build.')
 elif 'df_merged' not in globals():
     warnings.warn('df_merged is not defined; cannot project forward-looking label onto merged feature table.')
 else:
     df_health[date_col] = pd.to_datetime(df_health[date_col], errors='coerce')
     outbreak_series = df_health[outbreak_col].astype(str).str.strip().str.lower()
     valid_mask = ~outbreak_series.isin({'', 'none', 'null', 'nan'})
     df_events = df_health[valid_mask][['district', 'district_norm', date_col]].copy()
     df_events = df_events.rename(columns={date_col: 'report_date'})

     def has_future_outbreak(row):
         d_norm = row.get('district_norm')
         if d_norm is None or pd.isna(d_norm):
             d_norm = norm_dist(pd.Series([row.get('district', '')])).iloc[0]
         t0 = row.get('feature_date')
         if d_norm is None or pd.isna(t0):
             return 0
         ev = df_events[df_events['district_norm'] == d_norm]['report_date']
         if ev.empty:
             return 0
         future = ev[(ev > t0) & (ev <= t0 + pd.Timedelta(days=HORIZON_DAYS))]
         return int(not future.empty)

     df_merged['district_norm'] = norm_dist(df_merged['district']) if 'district_norm' not in df_merged.columns else df_merged['district_norm']
     df_merged['outbreak_future_14d'] = df_merged.apply(has_future_outbreak, axis=1)
     df_merged['outbreak_risk'] = df_merged['outbreak_future_14d']
     print(f"Created forward-looking label outbreak_future_14d using {date_col} / {outbreak_col}")

 # 2) Parse crime JSON into district-level columns (optional)
 if 'crime_reports' in df_safety.columns and 'df_merged' in globals():
     def _parse_crime_json(x):
         try:
             if pd.isna(x):
                 return {}
             if isinstance(x, str):
                 parsed = json.loads(x)
             else:
                 parsed = x
         except Exception:
             return {}
         counts = {}
         if isinstance(parsed, dict):
             parsed = [parsed]
         if isinstance(parsed, (list, tuple)):
             for item in parsed:
                 if not isinstance(item, dict):
                     continue
                 ctype = item.get('type')
                 cval = item.get('count', 1)
                 try:
                     cval = int(float(cval))
                 except Exception:
                     cval = 0
                 if ctype and cval:
                     counts[ctype] = counts.get(ctype, 0) + cval
         return counts

     crime_expanded = df_safety[['district_norm', 'crime_reports']].copy()
     crime_expanded['crime_counts'] = crime_expanded['crime_reports'].apply(_parse_crime_json)
     crime_cols = set()
     for counts in crime_expanded['crime_counts']:
         crime_cols.update(counts.keys())
     for crime in crime_cols:
         df_merged[f'crime_{crime}'] = df_merged['district_norm'].map(
             crime_expanded.set_index('district_norm')['crime_counts'].apply(lambda d: d.get(crime, 0) if isinstance(d, dict) else 0)
         )
     print(f'Merged crime JSON counts into df_merged ({len(crime_cols)} columns).')
 elif 'df_merged' not in globals():
     warnings.warn('df_merged is not defined; skipping crime_reports feature engineering.')
 else:
     print('No crime_reports column available to parse.')

 # 3) Placeholder for rolling environmental joins (requires df_merged with feature_date)
 env_date_col = find_date_col(df_env)
 if env_date_col and 'df_merged' in globals():
     df_env[env_date_col] = pd.to_datetime(df_env[env_date_col], errors='coerce')
     env_weekly = df_env.copy()
     env_weekly['week_start'] = norm_week(env_weekly[env_date_col])
     env_metrics = env_weekly.groupby(['district_norm', 'week_start']).agg({
         'pm25_level': 'mean',
         'pm10_level': 'mean',
         'air_quality_index': 'mean'
     }).reset_index()
     df_merged['feature_week'] = norm_week(df_merged['feature_date'])
     df_merged = df_merged.merge(
         env_metrics.rename(columns={'week_start': 'feature_week'}),
         on=['district_norm', 'feature_week'],
         how='left'
     )
     print('Merged weekly environmental metrics onto df_merged (demo).')
 elif 'df_merged' not in globals():
     warnings.warn('df_merged missing; skipping environmental feature join.')
 else:
     print('No date column in environment_monitoring; skipping environmental window features.')

 print('Horizon & windowed feature step complete (demo skeleton).')

KeyError: 'Disease Outbreak'

In [ ]:
# ------------------------------------------------------------------
# CELL 5: Preprocess Data (use df_model from district-week panel)
# ------------------------------------------------------------------

# Select limited, meaningful features for modeling (as-of at week_start)
relevant_features = [
    'pm25_level', 'pm10_level', 'air_quality_index', 'water_quality_index', 'waste_collection_efficiency',
    'population_density_per_sqkm', 'avg_household_size',
    'condition_score', 'traffic_volume_daily',
    'patient_inflow_mean',
    # rolling features
    'pm25_roll_1w', 'pm25_roll_2w', 'pm25_roll_4w',
    'patient_inflow_roll_1w','patient_inflow_roll_2w','patient_inflow_roll_4w'
]

# Ensure features exist (some env columns may not be present)
relevant_features = [f for f in relevant_features if f in df_model.columns]
print('Using features:', relevant_features)

# Target
target_col = 'outbreak_next_14d'
X_df = df_model[['district','week_start'] + relevant_features].copy()
y = df_model[target_col].copy()

# Drop initial rows with NaN in key features
X_df = X_df.dropna(subset=relevant_features)
# align y
y = y.loc[X_df.index]

# Split by time: train up to a date, test after
cutoff = X_df['week_start'].quantile(0.8)
train_mask = X_df['week_start'] <= cutoff
X_train_raw = X_df[train_mask].drop(columns=['district','week_start'])
X_test_raw = X_df[~train_mask].drop(columns=['district','week_start'])
y_train = y[train_mask]
y_test = y[~train_mask]

print(f'Train weeks: {X_df[train_mask]["week_start"].min()} to {X_df[train_mask]["week_start"].max()}')
print(f'Test weeks: {X_df[~train_mask]["week_start"].min()} to {X_df[~train_mask]["week_start"].max()}')

# Preprocessing: numeric only for now
numerical_cols = X_train_raw.select_dtypes(include=[np.number]).columns.tolist()

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
])

preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols)])

X_train = preprocessor.fit_transform(X_train_raw)
X_test = preprocessor.transform(X_test_raw)

print('Final feature matrix shapes:', X_train.shape, X_test.shape)
print('Train positive rate:', y_train.mean(), 'Test positive rate:', y_test.mean())


In [ ]:
# ------------------------------------------------------------------
# CELL 6: Data Exploration (Split moved to CELL 5)
# ------------------------------------------------------------------

print("Time-based data split completed in CELL 5 to avoid data leakage.")
print(f"Train period: {X_df[train_mask]['week_start'].min()} to {X_df[train_mask]['week_start'].max()}")
print(f"Test period: {X_df[~train_mask]['week_start'].min()} to {X_df[~train_mask]['week_start'].max()}")
print('Number of districts in panel:', df_model['district'].nunique())
print('Date range in panel:', df_model['week_start'].min(), 'to', df_model['week_start'].max())


In [ ]:
# ------------------------------------------------------------------
# CELL 7: Choose Best Algorithm (class-weighting / scale_pos_weight) + Calibration
# ------------------------------------------------------------------

from sklearn.metrics import average_precision_score
from sklearn.calibration import CalibratedClassifierCV

# Split training into train_inner / val for calibration (last 10% of training as val)
n_train = len(X_train)
val_n = max(1, int(0.1 * n_train))
train_inner_n = n_train - val_n
X_train_inner = X_train[:train_inner_n]
X_val = X_train[train_inner_n:]
y_train_inner = y_train.iloc[:train_inner_n]
y_val = y_train.iloc[train_inner_n:]

# Compute class weight / scale
pos = y_train.sum()
neg = len(y_train) - pos
scale_pos_weight = (neg / pos) if pos>0 else 1.0
print('scale_pos_weight (neg/pos):', scale_pos_weight)

models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    'XGBoost': xgb.XGBClassifier(objective='binary:logistic', scale_pos_weight=scale_pos_weight, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)
}

best_model = None
best_auc = 0
best_name = ''

for name, model in models.items():
    # Fit on inner training
    model.fit(X_train_inner, y_train_inner)
    # Calibrate on val using prefit
    try:
        calib = CalibratedClassifierCV(model, cv='prefit', method='sigmoid')
        calib.fit(X_val, y_val)
        model_to_eval = calib
    except Exception:
        model_to_eval = model
    # Evaluate on test
    y_pred = model_to_eval.predict(X_test)
    y_pred_proba = model_to_eval.predict_proba(X_test)[:, 1]
    f1 = f1_score(y_test, y_pred, average='weighted')
    auc = roc_auc_score(y_test, y_pred_proba)
    pr_auc = average_precision_score(y_test, y_pred_proba)
    brier = brier_score_loss(y_test, y_pred_proba)
    print(f"{name} - F1-Score: {f1:.4f}, ROC AUC: {auc:.4f}, PR AUC: {pr_auc:.4f}, Brier: {brier:.4f}")
    print(classification_report(y_test, y_pred))

    # recall@K: top-K by predicted probability (e.g., top 10% weeks)
    k = max(1, int(0.1 * len(y_test)))
    topk_idx = np.argsort(y_pred_proba)[-k:]
    recall_at_k = y_test.iloc[topk_idx].sum() / y_test.sum() if y_test.sum()>0 else 0.0
    print(f'Recall@{k} (top 10%): {recall_at_k:.3f}')

    if auc > best_auc:
        best_auc = auc
        best_model = model
        best_name = name

print(f"\nBest model: {best_name} with ROC AUC {best_auc:.4f}")

# Retrain best model on full training data and calibrate using last 10% of original training
best_model.fit(X_train, y_train)
calibrated = None
try:
    val_for_cal = X_train[-val_n:]
    y_for_cal = y_train.iloc[-val_n:]
    calibrated = CalibratedClassifierCV(best_model, cv='prefit', method='sigmoid')
    calibrated.fit(val_for_cal, y_for_cal)
    final_model = calibrated
    print('Calibration applied to final model.')
except Exception as e:
    final_model = best_model
    print('Calibration skipped (fallback to raw model):', e)

# Feature importance for tree models
if hasattr(best_model, 'feature_importances_'):
    try:
        # try to recover feature names from numerical_cols
        feature_names = numerical_cols
    except Exception:
        feature_names = [f'feat_{i}' for i in range(X_train.shape[1])]
    importances = best_model.feature_importances_
    feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
    feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)
    print("\nTop 10 Feature Importances:")
    print(feature_importance_df.head(10))

# Save final_model reference for downstream cells
best_model = final_model


In [ ]:
# ------------------------------------------------------------------
# CELL 8: Train Final Model and Save (artifact at root) 
# ------------------------------------------------------------------

# Create full pipeline with preprocessor and classifier
final_clf = type(best_model)(**best_model.get_params())
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', final_clf)
])

# Fit on full training set (X_train_raw aligned earlier)
full_pipeline.fit(X_train_raw, y_train)

# Save artifact at repo root with standard name expected by sklearn container
import joblib
joblib.dump(full_pipeline, 'model.joblib')
print('Saved pipeline as model.joblib')

# ------------------------------------------------------------------
# CELL 9: Upload to GCS and Deploy Vertex AI Endpoint (demo)
# ------------------------------------------------------------------

# Require user to set a BUCKET variable
BUCKET = 'your-bucket'  # <-- Replace this with your GCS bucket name or set via environment
if BUCKET == 'your-bucket':
    print('Please set BUCKET to your real GCS bucket before running deployment steps. Skipping upload/deploy.')
else:
    # Upload to GCS root of model-artifacts
    !gsutil cp model.joblib gs://{BUCKET}/model.joblib

    # Upload to Vertex AI and deploy
    model = aiplatform.Model.upload(
        display_name=MODEL_DISPLAY_NAME,
        artifact_uri=f'gs://{BUCKET}/',
        serving_container_image_uri='us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest',
        project=PROJECT_ID,
        location=REGION
    )

    endpoint = model.deploy(machine_type='n1-standard-2', min_replica_count=1, max_replica_count=1)
    print(f'Deployed model to endpoint: {endpoint.resource_name}')
